<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [38]:
%load_ext autoreload 
%autoreload 2
%matplotlib inline 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SVMSMOTE 

import torch
from torchdata.datapipes.iter import FileLister, Zipper
from torch.utils.data import DataLoader

import numpy as np

import my_data_pipes

In [45]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)
device = torch.device(device)

cpu


In [41]:
svm_smote = SVMSMOTE(sampling_strategy=0.8, random_state=42)
imputer = SimpleImputer() 
scale = StandardScaler()

In [28]:

def reshaper(seq): 
    return np.array(seq).reshape(1,-1)

def feature_label_split(dp, sequence_length): 
    *feature_dp, label_dp = dp.unzip(sequence_length=sequence_length)
    return Zipper(*feature_dp).map(reshaper), label_dp

def apply_balancer(feature_dp, label_dp, balancer): 
    return Zipper(feature_dp, label_dp).balance(balancer) 

datapipe = FileLister(root='data', ).open_files(mode='t').read_arff().shuffle(buffer_size=5000)
datapipe = datapipe.map(float, input_col='class')
train, test = datapipe.random_split(weights={'train': 0.8, 'test':0.2}, seed=42, total_length=43405)

train_feature_dp, train_label_dp = feature_label_split(train, sequence_length=65)

X= DataLoader(train_feature_dp, batch_size=5000) 
X= next(iter(X))
X= X.reshape(5000, 64)

imputer.fit(X) 
scale.fit(X) 

train_feature_dp = train_feature_dp.map(imputer.transform).map(scale.transform)
balanced_train = apply_balancer(train_feature_dp, train_label_dp, svm_smote)

train_dataloader = DataLoader(balanced_train, batch_size=32, shuffle=True)

test_feature_dp, test_label_dp = feature_label_split(test, sequence_length=65)
test_dataloader = DataLoader(test, batch_size=32)

In [29]:
from my_model import SimpleNet

In [32]:
from torch.nn import BCELoss
from torch.optim import Adam

net = SimpleNet(64, 10, 1).to(device)

loss_fn = BCELoss()
optimizer = Adam(net.parameters(), lr=0.05, weight_decay=0.1 ) 

In [33]:
MAX_EPOCHS = 2
for epoch in range(MAX_EPOCHS): 
    for it, (batch, labels) in enumerate(train_dataloader): 
        output = net(batch) 
        loss = loss_fn(output, labels) 

        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()

        if it % 200 == 0:
            loss = loss.item()
            print(f'Iter: {it: >4}  Loss:{loss: >0.12f}')
            
    recall, precis = 0, 0
    with torch.no_grad(): 
        for batch, labels in test_dataloader: 
            pred = net(batch) 
            pred = (pred >= 0.5).type(torch.float64)
            recall += (pred[labels==1] == labels[labels==1]).type(torch.float64).sum().item()
            precis += (pred[pred==1] == labels[pred==1]).type(torch.float64).sum().item()
    
    recall /= len(test_dataloader)
    precis /= len(test_dataloader)
    print(f"Recall: {recall: >0.3f} Precision: {precis: >0.3f}")



  0%|          | 0/2172 [00:00<?, ?it/s]

Iter:    0  Loss:0.846033183566
Iter:  200  Loss:0.132341444050
Iter:  400  Loss:0.232565873371


KeyboardInterrupt: 